# Creating and Editing Excel Files from Teradata

In [1]:
# Import libraries
import os
import teradata
import pandas as pd
import getpass
import numpy as np
import warnings
warnings.filterwarnings("ignore")

# Connect to Teradata

In [3]:
user = 'grumpy_statistician'
auth ='LDAP'
passw = getpass.getpass()

········


In [4]:
udaExec = teradata.UdaExec(appName="test", version="2.0", logConsole=False)
session = udaExec.connect(method="odbc", system='YOUR_SERVER',
                          username=user, password=passw, authentication=auth,
                          driver="Teradata Database ODBC Driver")

# Write Query/Queries 
Refer to Teradata Module.ipynb for more information

In [7]:
# for tab 1

df = pd.read_sql('''
SELECT
*
FROM YOUR_DB.YOUR_TABLE

''',session)


df2 = pd.read_sql('''
SELECT
*
FROM YOUR_DB.YOUR_TABLE

''',session)



# Example: Create two tab Excel file

In [10]:
# https://xlsxwriter.readthedocs.io/index.html for formatting tips

writer = pd.ExcelWriter(f'your_filename.xlsx', engine='xlsxwriter')

# tab 1
df.to_excel(writer, sheet_name='your_sheet_name', index=False)
#want to replace 0s with Nan for PY22_CLM 
df2['PY22_STUFF'] = df2['PY22_STUFF'].replace(0,np.nan)
# tab 2
df2.to_excel(writer, sheet_name='PY STUFF', index=False)
#create workbook
workbook = writer.book

# Cell and Row Formatting for Table 

In [11]:
# Create worksheet for tab1
worksheet1 = writer.sheets['your_sheet_name']

# Set first row formatting
row_all = workbook.add_format({'bold': True, 'align': 'center'})
worksheet1.set_row(0, 15, row_all)  # 15 is the column width

# Set Column formatting
column_all = workbook.add_format({ 'align': 'left'})
worksheet1.set_column('A:C', 35, column_all)

# Add Commas to Claim IDs
num_format = workbook.add_format({'num_format': '#,##0'})
worksheet1.set_column('C:C', 25, num_format)

# Add currency format to Paid
currency_format = workbook.add_format({'num_format': '$#,##0.00'})
worksheet1.set_column('B:B', 25, currency_format)

0

# Creating, Formatting, and Inserting Line Chart 

In [12]:
# reference https://xlsxwriter.readthedocs.io/chart.html

chart = workbook.add_chart({'type': 'line'})
chart.add_series({
    'name': 'Total Paid',
    'categories': '=your_sheet_name!A2:A22', #Categories are the dates in this case, treat the df like an Excel file
    'values': '=CCSR!B2:B22' 
    })
# formatting
chart.set_title({'name': 'Paid by Stuff'})

chart.set_x_axis({'name': 'Date'})

chart.set_y_axis({'name': 'Paid Amounts', 'major_gridlines': {'visible': False}})

chart.set_legend({'none': True})

# Set an Excel chart style.
chart.set_style(5)

# Insert
worksheet1.insert_chart('E1', chart)


# Creating, Formatting, and Inserting Column Chart 

In [13]:
# Create
chart2 = workbook.add_chart({'type': 'column'})
chart2.add_series({
    'name': 'Total Stuff IDs',
    'categories': '=your_sheet_name!A2:A22', #Categories are the dates in this case, treat the df like an Excel file
    'values': '=your_sheet_name!C2:C22' 
    })
# formatting
chart2.set_title({'name': 'Stuff by Stuff'})

chart2.set_x_axis({'name': 'Date'})

chart2.set_y_axis({'name': 'Claim IDs', 'major_gridlines': {'visible': False}})

chart2.set_legend({'none': True})

# Set an Excel chart style.
chart2.set_style(6)

# Insert
worksheet1.insert_chart('E16', chart2)

# Cell and Row Formatting 

In [14]:
# Create worksheet for tab2
worksheet2 = writer.sheets['PY Stuff & Stuff']

# Set first row formatting
worksheet2.set_row(0, 15, row_all)  # 15 is the column width

# Set Column formatting
worksheet2.set_column('A:C', 35, column_all)

# Add Commas to Claim IDs
worksheet2.set_column('D:E', 25, num_format)

# Add currency format to Paid
worksheet2.set_column('B:C', 25, currency_format)



0

# Creating, Formatting, and Inserting Line Chart w/ Two Categories 

In [15]:
# Create

chart3 = workbook.add_chart({'type': 'line'})
chart3.add_series({
         'name':      "'PY Stuff & Stuff'!B1",
        'categories': "'PY Stuff & Stuffs'!A2:A13",
        'values':     "'PY Stuff & Stuff'!B2:B13",
    })
chart3.add_series({
         'name':      "'PY Stuff & Stuff'!C1",
        'categories': "'PY Stuff & Stuff'!A2:A13",
        'values':     "'PY Stuff & Stuff'!C2:C13",
    })


# formatting
chart3.set_title({'name': 'Paid by Stuff & Stuff'})

chart3.set_x_axis({'name': 'Month'})

chart3.set_y_axis({'name': 'Amounts', 'major_gridlines': {'visible': False}})

#chart3.set_legend({'none': True})

# Set an Excel chart style.
chart3.set_style(7)

# Insert
worksheet2.insert_chart('A15', chart3)

# Creating, Formatting, and Inserting Column Chart w/ Two Categories Part 2

In [16]:
# Create

chart4 = workbook.add_chart({'type': 'column'})
chart4.add_series({
         'name':      "'PY Stuff & Stuff'!D1",
        'categories': "'PY Stuff & Stuff'!A2:A13",
        'values':     "'PY Stuff & Stuff'!D2:D13",
    })
chart4.add_series({
         'name':      "'PY Stuff & Stuff'!E1",
        'categories': "'PY Stuff & Stuff'!A2:A13",
        'values':     "'PY Stuff & Stuff'!E2:E13",
    })

# formatting
chart4.set_title({'name': 'Stuff by Plan Year and Month'})

chart4.set_x_axis({'name': 'Month'})

chart4.set_y_axis({'name': 'Stuff', 'major_gridlines': {'visible': False}})

#chart3.set_legend({'none': True})

# Set an Excel chart style.
chart4.set_style(22)

# Insert
worksheet2.insert_chart('D15', chart4)

Save file

In [17]:
writer.save()